## Masked-LLM 掩码语言模型训练实例

### Step1 导入相关包

In [2]:
import os

# 设置可见的 GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "4,5,7"

from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer

### Step2 加载数据集

In [3]:
ds = load_dataset("pleisto/wikipedia-cn-20230720-filtered")
ds

DatasetDict({
    train: Dataset({
        features: ['completion', 'source'],
        num_rows: 254547
    })
})

In [4]:
ds["train"][0]

{'completion': '昭通机场（ZPZT）是位于中国云南昭通的民用机场，始建于1935年，1960年3月开通往返航班“昆明－昭通”，原来属军民合用机场。1986年机场停止使用。1991年11月扩建，于1994年2月恢复通航。是西南地区「文明机场」，通航城市昆明。 机场占地1957亩，飞行区等级为4C，有一条跑道，长2720米，宽48米，可供波音737及以下机型起降。机坪面积6600平方米，停机位2个，航站楼面积1900平方米。位于城东6公里处，民航路与金鹰大道交叉处。\n航点\n客服电话\n昭通机场客服电话：0870-2830004',
 'source': 'wikipedia.zh2307'}

### Step3 数据集处理

In [5]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

In [6]:
def process_function(examples):
    return tokenizer(examples["completion"], truncation=True, max_length=384)

In [7]:
tokenized_ds = ds.map(process_function, batched=True, remove_columns=ds["train"].column_names)
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 254547
    })
})

In [8]:
from torch.utils.data import DataLoader

dl = DataLoader(tokenized_ds["train"], batch_size=2, collate_fn=DataCollatorForLanguageModeling(tokenizer, mlm=True, mlm_probability=0.15))
dl

In [9]:
next(enumerate(dl))  # attention_mask中的非-100值代表被mask的token，即input_ids里的103

(0,
 {'input_ids': tensor([[  101,  3220,  6858,  3322,  1767,  8020,   168,  8187,  8253,  8165,
           8021,  3221,   855,   754,   704,  1744,   756,  1298,  3220,  6858,
           4638,  8817,  4500,  3322, 10592,  8024,  1993,  2456,   754,  9523,
           2399,  8024,  8779,  2399,   124,  3299,  2458,  6858,  2518,  6819,
           5661,  4408,   100,  3204,  3209,  8025,  3220,  6858,   100,   103,
           1333,  3341,  2247,  1092,  3696,  1394,  4500,  3322,  1767,   511,
           8629,  2399,   103,  1767,   977,  3632,   886,  4500,   511,  8555,
           2399,  8111,  3299,  2810,  2456,  8024,   754,  8447,  2399,   103,
           3299,  2612,  1908,  6858,  5661,   511,  3221,  6205,  1298,  1765,
           4662,   519,  3152,  3209,  3322,   103,   520,  8024,  6858,  5661,
           1814,   103,  3204,   103,   511,  2865,  1767,  1304,  1765,  9088,
            774,  8024,   103,  6121,  1277,  5023,  5277,   711,   103,  8177,
           8024,  3300

In [10]:
tokenizer.mask_token, tokenizer.mask_token_id

('[MASK]', 103)

### Step4 创建模型

In [11]:
model = AutoModelForMaskedLM.from_pretrained("hfl/chinese-macbert-base")

Some weights of the model checkpoint at hfl/chinese-macbert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Step5 配置训练参数

In [12]:
import logging

logging.basicConfig(level=logging.INFO)

In [13]:
args = TrainingArguments(
    output_dir="./masked_lm",
    per_device_train_batch_size=32,
    logging_steps=10,
    num_train_epochs=1
)

### Step6 创建训练器

In [14]:
trainer = Trainer(
    args=args,
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds["train"],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=True, mlm_probability=0.15)
)

### Step7 模型训练

In [15]:
trainer.train()

/node6_1/tanshuai/.conda/envs/abc/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
10,1.388500
20,1.363000
30,1.330500
40,1.327900
50,1.336100
60,1.326500
70,1.289100
80,1.318600
90,1.307300
100,1.337200


/node6_1/tanshuai/.conda/envs/abc/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/node6_1/tanshuai/.conda/envs/abc/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/node6_1/tanshuai/.conda/envs/abc/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/node6_1/tanshuai/.conda/envs/abc/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: W

TrainOutput(global_step=2652, training_loss=1.2153524243094442, metrics={'train_runtime': 2272.3668, 'train_samples_per_second': 112.018, 'train_steps_per_second': 1.167, 'total_flos': 5.024298725460173e+16, 'train_loss': 1.2153524243094442, 'epoch': 1.0})

### Step8 模型推理

In [17]:
from transformers import pipeline

pipe = pipeline("fill-mask", model=model, tokenizer=tokenizer, device=0)

In [18]:
pipe("昭通机场（ZPZT）是位于中国云南昭通的民用机[MASK]，始建于1935年，1960年3月开通往返航班“昆明－昭通”，原来属军民合用机场。1986年机场停止使用。1991年11月扩建，于1994年2月恢复通航。是西南地区「文明机场」，通航城市昆明。 机场占地1957亩，飞行区等级为4C，有一条跑道，长2720米，宽48米，可供波音737及以下机型起降。机坪面积6600平方米，停机位2个，航站楼面积1900平方米。位于城东6公里处，民航路与金鹰大道交叉处。\n航点\n客服电话\n昭通机场客服电话：0870-2830004")

[{'score': 0.9998067021369934,
  'token': 1767,
  'token_str': '场',
  'sequence': '昭 通 机 场 （ zpzt ） 是 位 于 中 国 云 南 昭 通 的 民 用 机 场 ， 始 建 于 1935 年 ， 1960 年 3 月 开 通 往 返 航 班 昆 明 － 昭 通 ， 原 来 属 军 民 合 用 机 场 。 1986 年 机 场 停 止 使 用 。 1991 年 11 月 扩 建 ， 于 1994 年 2 月 恢 复 通 航 。 是 西 南 地 区 「 文 明 机 场 」 ， 通 航 城 市 昆 明 。 机 场 占 地 1957 亩 ， 飞 行 区 等 级 为 4c ， 有 一 条 跑 道 ， 长 2720 米 ， 宽 48 米 ， 可 供 波 音 737 及 以 下 机 型 起 降 。 机 坪 面 积 6600 平 方 米 ， 停 机 位 2 个 ， 航 站 楼 面 积 1900 平 方 米 。 位 于 城 东 6 公 里 处 ， 民 航 路 与 金 鹰 大 道 交 叉 处 。 航 点 客 服 电 话 昭 通 机 场 客 服 电 话 ： 0870 - 2830004'},
 {'score': 0.00011264372005825862,
  'token': 4991,
  'token_str': '站',
  'sequence': '昭 通 机 场 （ zpzt ） 是 位 于 中 国 云 南 昭 通 的 民 用 机 站 ， 始 建 于 1935 年 ， 1960 年 3 月 开 通 往 返 航 班 昆 明 － 昭 通 ， 原 来 属 军 民 合 用 机 场 。 1986 年 机 场 停 止 使 用 。 1991 年 11 月 扩 建 ， 于 1994 年 2 月 恢 复 通 航 。 是 西 南 地 区 「 文 明 机 场 」 ， 通 航 城 市 昆 明 。 机 场 占 地 1957 亩 ， 飞 行 区 等 级 为 4c ， 有 一 条 跑 道 ， 长 2720 米 ， 宽 48 米 ， 可 供 波 音 737 及 以 下 机 型 起 降 。 机 坪 面 积 6600 平 方 米 ， 停 机 位 2 个 ， 航 站 楼 面 积 1900 平 方 米 。 位 于 城 

In [21]:
pipe("昭通机场（ZPZT）是位于中国云南昭通的民用[MASK]，始建于1935年，1960年3月开通往返航班“昆明－昭通”，原来属军民合用机场。")

[{'score': 0.7497851252555847,
  'token': 1767,
  'token_str': '场',
  'sequence': '昭 通 机 场 （ zpzt ） 是 位 于 中 国 云 南 昭 通 的 民 用 场 ， 始 建 于 1935 年 ， 1960 年 3 月 开 通 往 返 航 班 昆 明 － 昭 通 ， 原 来 属 军 民 合 用 机 场 。'},
 {'score': 0.18158943951129913,
  'token': 3322,
  'token_str': '机',
  'sequence': '昭 通 机 场 （ zpzt ） 是 位 于 中 国 云 南 昭 通 的 民 用 机 ， 始 建 于 1935 年 ， 1960 年 3 月 开 通 往 返 航 班 昆 明 － 昭 通 ， 原 来 属 军 民 合 用 机 场 。'},
 {'score': 0.034680455923080444,
  'token': 11435,
  'token_str': 'airport',
  'sequence': '昭 通 机 场 （ zpzt ） 是 位 于 中 国 云 南 昭 通 的 民 用 airport ， 始 建 于 1935 年 ， 1960 年 3 月 开 通 往 返 航 班 昆 明 － 昭 通 ， 原 来 属 军 民 合 用 机 场 。'},
 {'score': 0.006334721576422453,
  'token': 4991,
  'token_str': '站',
  'sequence': '昭 通 机 场 （ zpzt ） 是 位 于 中 国 云 南 昭 通 的 民 用 站 ， 始 建 于 1935 年 ， 1960 年 3 月 开 通 往 返 航 班 昆 明 － 昭 通 ， 原 来 属 军 民 合 用 机 场 。'},
 {'score': 0.006037409417331219,
  'token': 7252,
  'token_str': '镇',
  'sequence': '昭 通 机 场 （ zpzt ） 是 位 于 中 国 云 南 昭 通 的 民 用 镇 ， 始 建 于 1935 年 ， 1960 年 3 月 开 通 往 返 航 班 昆 明 － 昭 通 

In [22]:
pipe("昭通机场（ZPZT）是位于中国云南昭通的民用[MASK][MASK]，始建于1935年，1960年3月开通往返航班“昆明－昭通”，原来属军民合用机场。")

[[{'score': 0.9981327652931213,
   'token': 3322,
   'token_str': '机',
   'sequence': '[CLS] 昭 通 机 场 （ zpzt ） 是 位 于 中 国 云 南 昭 通 的 民 用 机 [MASK] ， 始 建 于 1935 年 ， 1960 年 3 月 开 通 往 返 航 班 [UNK] 昆 明 － 昭 通 [UNK] ， 原 来 属 军 民 合 用 机 场 。 [SEP]'},
  {'score': 0.0008135749376378953,
   'token': 5661,
   'token_str': '航',
   'sequence': '[CLS] 昭 通 机 场 （ zpzt ） 是 位 于 中 国 云 南 昭 通 的 民 用 航 [MASK] ， 始 建 于 1935 年 ， 1960 年 3 月 开 通 往 返 航 班 [UNK] 昆 明 － 昭 通 [UNK] ， 原 来 属 军 民 合 用 机 场 。 [SEP]'},
  {'score': 0.0005287771346047521,
   'token': 7607,
   'token_str': '飞',
   'sequence': '[CLS] 昭 通 机 场 （ zpzt ） 是 位 于 中 国 云 南 昭 通 的 民 用 飞 [MASK] ， 始 建 于 1935 年 ， 1960 年 3 月 开 通 往 返 航 班 [UNK] 昆 明 － 昭 通 [UNK] ， 原 来 属 军 民 合 用 机 场 。 [SEP]'},
  {'score': 0.00013867772941011935,
   'token': 4958,
   'token_str': '空',
   'sequence': '[CLS] 昭 通 机 场 （ zpzt ） 是 位 于 中 国 云 南 昭 通 的 民 用 空 [MASK] ， 始 建 于 1935 年 ， 1960 年 3 月 开 通 往 返 航 班 [UNK] 昆 明 － 昭 通 [UNK] ， 原 来 属 军 民 合 用 机 场 。 [SEP]'},
  {'score': 8.497361704939976e-05,
   'token':

In [23]:
pipe("昭通机场（ZPZT）是位于中国云南昭通的民用机[MASK]，始建于1935年，1960年3月开通往返航[MASK]“昆明－昭通”，原来属军民合用机场。")

[[{'score': 0.9999030828475952,
   'token': 1767,
   'token_str': '场',
   'sequence': '[CLS] 昭 通 机 场 （ zpzt ） 是 位 于 中 国 云 南 昭 通 的 民 用 机 场 ， 始 建 于 1935 年 ， 1960 年 3 月 开 通 往 返 航 [MASK] [UNK] 昆 明 － 昭 通 [UNK] ， 原 来 属 军 民 合 用 机 场 。 [SEP]'},
  {'score': 4.1764284105738625e-05,
   'token': 3354,
   'token_str': '构',
   'sequence': '[CLS] 昭 通 机 场 （ zpzt ） 是 位 于 中 国 云 南 昭 通 的 民 用 机 构 ， 始 建 于 1935 年 ， 1960 年 3 月 开 通 往 返 航 [MASK] [UNK] 昆 明 － 昭 通 [UNK] ， 原 来 属 军 民 合 用 机 场 。 [SEP]'},
  {'score': 1.568646439409349e-05,
   'token': 1790,
   'token_str': '坪',
   'sequence': '[CLS] 昭 通 机 场 （ zpzt ） 是 位 于 中 国 云 南 昭 通 的 民 用 机 坪 ， 始 建 于 1935 年 ， 1960 年 3 月 开 通 往 返 航 [MASK] [UNK] 昆 明 － 昭 通 [UNK] ， 原 来 属 军 民 合 用 机 场 。 [SEP]'},
  {'score': 5.9489611885510385e-06,
   'token': 1842,
   'token_str': '場',
   'sequence': '[CLS] 昭 通 机 场 （ zpzt ） 是 位 于 中 国 云 南 昭 通 的 民 用 机 場 ， 始 建 于 1935 年 ， 1960 年 3 月 开 通 往 返 航 [MASK] [UNK] 昆 明 － 昭 通 [UNK] ， 原 来 属 军 民 合 用 机 场 。 [SEP]'},
  {'score': 3.592073426261777e-06,
   'token'

In [24]:
pipe("下面是一则[MASK][MASK]新闻。小编报道，近日，游戏产业发展的非常好！")

[[{'score': 0.07495887577533722,
   'token': 2031,
   'token_str': '娱',
   'sequence': '[CLS] 下 面 是 一 则 娱 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.06764405220746994,
   'token': 3952,
   'token_str': '游',
   'sequence': '[CLS] 下 面 是 一 则 游 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.04328230395913124,
   'token': 5381,
   'token_str': '网',
   'sequence': '[CLS] 下 面 是 一 则 网 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.0352151021361351,
   'token': 6568,
   'token_str': '财',
   'sequence': '[CLS] 下 面 是 一 则 财 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.025313353165984154,
   'token': 7028,
   'token_str': '重',
   'sequence': '[CLS] 下 面 是 一 则 重 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'}],
 [{'score': 0.07272625714540482,
   'token': 5317,
   'token_str': '络',
   'sequence': '[CLS] 下 面 是 一 则 [MASK] 络 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {

In [31]:
pipe("结构化剪[MASK]是一种神经网络模型压缩算法。")

[{'score': 0.13429778814315796,
  'token': 6782,
  'token_str': '辑',
  'sequence': '结 构 化 剪 辑 是 一 种 神 经 网 络 模 型 压 缩 算 法 。'},
 {'score': 0.06307534873485565,
  'token': 2970,
  'token_str': '接',
  'sequence': '结 构 化 剪 接 是 一 种 神 经 网 络 模 型 压 缩 算 法 。'},
 {'score': 0.06147187948226929,
  'token': 6585,
  'token_str': '贴',
  'sequence': '结 构 化 剪 贴 是 一 种 神 经 网 络 模 型 压 缩 算 法 。'},
 {'score': 0.05061841011047363,
  'token': 4772,
  'token_str': '码',
  'sequence': '结 构 化 剪 码 是 一 种 神 经 网 络 模 型 压 缩 算 法 。'},
 {'score': 0.04697728529572487,
  'token': 1143,
  'token_str': '刀',
  'sequence': '结 构 化 剪 刀 是 一 种 神 经 网 络 模 型 压 缩 算 法 。'}]